# Exercise 1.3

## Classification of CIFAR10 images
### Optimizers
In this exercise we will classify the images from the CIFAR10 dataset. We will use different optimizers and compare their convergence speed. First we import the libraries that we need.

In [7]:
import random
import numpy as np
from tqdm.notebook import tqdm
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

We always check that we are running on a GPU

In [8]:
def set_seed(seed=1337):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(1337)
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In this exercise we will classify images from the [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset. 
CIFAR10 has 60000 colour images of size 32x32 equally distributed in 10 classes.
* You should load this dataset (hint: it is a built-in dataset in pytorch).

In [9]:

trainset_for_mean_std = datasets.CIFAR10(root="./data", train=True, download=True,
                            transform=transforms.ToTensor())

train_loader_for_mean_std = torch.utils.data.DataLoader(trainset_for_mean_std, batch_size=50000, shuffle=False)

data = next(iter(train_loader_for_mean_std))[0]  # shape [50000, 3, 32, 32]
mean = data.mean(dim=[0,2,3])
std  = data.std(dim=[0,2,3])

print("mean:", mean)
print("std:", std)


train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

batch_size = 96

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_tf)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_tf)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# Sanity batch
images, labels = next(iter(train_loader))
print("Batch:", images.shape, labels.shape)   # expect [128, 3, 32, 32], [128]
print("dtype:", images.dtype, "min/max:", images.min().item(), images.max().item())

mean: tensor([0.4914, 0.4822, 0.4465])
std: tensor([0.2470, 0.2435, 0.2616])
Batch: torch.Size([96, 3, 32, 32]) torch.Size([96])
dtype: torch.float32 min/max: -1.9892172813415527 2.126786708831787


* Make a CNN to train on the CIFAR10 dataset

In [10]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels: int, growth_rate: int, drop_rate: float = 0.0):
        super(DenseLayer, self).__init__()
        inter_channels = 4 * growth_rate  # bottleneck width

        self.norm1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, inter_channels, kernel_size=1, bias=False)

        self.norm2 = nn.BatchNorm2d(inter_channels)
        self.conv2 = nn.Conv2d(inter_channels, growth_rate, kernel_size=3, padding=1, bias=False)

        self.drop_rate = drop_rate
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv1(F.relu(self.norm1(x), inplace=True))
        out = self.conv2(F.relu(self.norm2(out), inplace=True))
        if self.drop_rate > 0.0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        # return only the new features (k channels)
        return out
    
class DenseBlock(nn.Module):
    def __init__(self, num_layers: int, in_channels: int, growth_rate: int, drop_rate: float = 0.0):
        super().__init__()
        layers = []
        channels = in_channels
        for i in range(num_layers):
            layers.append(DenseLayer(channels, growth_rate, drop_rate))
            channels += growth_rate
        self.layers = nn.ModuleList(layers)
        self.out_channels = channels  # convenience

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = [x]
        for layer in self.layers:
            new_feats = layer(torch.cat(features, dim=1))
            features.append(new_feats)
        return torch.cat(features, dim=1)  # in + L*k
    
class Transition(nn.Module):
    def __init__(self, in_channels: int, compression: float = 0.5):
        super().__init__()
        out_channels = int(in_channels * compression)
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.out_channels = out_channels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv(self.relu(self.norm(x)))
        out = self.pool(out)
        return out


class DenseNetCIFAR(nn.Module):
    def __init__(
        self,
        growth_rate: int = 24,
        block_layers=(16, 16, 16),
        compression: float = 0.5,
        stem_channels: int = 16,
        num_classes: int = 10,
        drop_rate: float = 0.0,
    ):
        super().__init__()
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate

        # Stem
        self.stem = nn.Conv2d(3, stem_channels, kernel_size=3, stride=1, padding=1, bias=False)

        # Block 1
        self.block1 = DenseBlock(block_layers[0], stem_channels, growth_rate, drop_rate)
        c1 = self.block1.out_channels
        self.trans1 = Transition(c1, compression)
        c1p = self.trans1.out_channels  # after compression

        # Block 2
        self.block2 = DenseBlock(block_layers[1], c1p, growth_rate, drop_rate)
        c2 = self.block2.out_channels
        self.trans2 = Transition(c2, compression)
        c2p = self.trans2.out_channels

        # Block 3
        self.block3 = DenseBlock(block_layers[2], c2p, growth_rate, drop_rate)
        c3 = self.block3.out_channels

        # Head
        self.norm = nn.BatchNorm2d(c3)
        self.relu = nn.ReLU(inplace=True)
        self.classifier = nn.Linear(c3, num_classes)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        x = self.stem(x)                     # B, C0, 32, 32
        x = self.block1(x)                   # channels grow to C0 + L1*k
        x = self.trans1(x)                   # halve channels, 32→16
        x = self.block2(x)
        x = self.trans2(x)                   # halve channels, 16→8
        x = self.block3(x)
        x = self.relu(self.norm(x))
        x = F.adaptive_avg_pool2d(x, 1).flatten(1)  # B, C3
        return self.classifier(x)


In [11]:
model = DenseNetCIFAR()
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [12]:
#We define the training as a function so we can easily re-use it.
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        return F.nll_loss(torch.log(output), target)
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            data, target = data.to(device), target.to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(data)
            #Compute the loss
            loss = loss_fun(output, target)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (target==predicted).sum().cpu().item()
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
        out_dict['train_acc'].append(train_correct/len(trainset))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
    return out_dict

 * Train the network and plot make a plot of the loss and accuracy for both training and with the epoch on the x-axis

In [13]:
out_dict = train(model, optimizer)
print(out_dict)
plt.plot()
plt.legend(('Test error','Train eror'))
plt.xlabel('Epoch number')
plt.ylabel('Accuracy')

  0%|          | 0/10 [00:00<?, ?epoch/s]

  0%|          | 0/521 [00:00<?, ?it/s]

KeyboardInterrupt: 

* Discuss what you see. Are you overfitting to the training data? Do you not learn anything? What can you change to do better?

* Repeat the above steps but using Adam as the optimizer. Use Pytorch's defaults parameters. Do you learn faster?
* Which optimizer works best for you?
* Plot the test and test errors for both SGD and Adam in one plot
* Try adding Batch normalisation after your convolutional layers. Does it help?

## ResNet

Now you will create and train a ResNet.
* Implement the Residual block as a network below using convolutional kernel size $3\times3$ according to the figure below
![Residual block](https://cdn-images-1.medium.com/max/800/1*D0F3UitQ2l5Q0Ak-tjEdJg.png)

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, n_features):
        super(ResNetBlock, self).__init__()
        ...
    def forward(self, x):
        ...
        return out

The following code is a sanity of your residual block network

In [ ]:
#Sanity test of your implementation
C = 4
res_block = ResNetBlock(C)
assert(len(res_block.state_dict())==4)
for name, weight in res_block.state_dict().items():
    weight*=0
    desired_shape = {'bias': (C,), 'weight': (C, C, 3, 3)}[name.split('.')[-1]]
    assert(desired_shape==weight.shape)
x = torch.randn(32, C, 32,32)
assert(torch.abs(res_block(x)-F.relu(x)).max()==0)
print("Passed sanity check")

We define a network that uses your `ResNetBlock`

In [ ]:
class ResNet(nn.Module):
    def __init__(self, n_in, n_features, num_res_blocks=3):
        super(ResNet, self).__init__()
        #First conv layers needs to output the desired number of features.
        conv_layers = [nn.Conv2d(n_in, n_features, kernel_size=3, stride=1, padding=1),
                       nn.ReLU()]
        for i in range(num_res_blocks):
            conv_layers.append(ResNetBlock(n_features))
        self.res_blocks = nn.Sequential(*conv_layers)
        self.fc = nn.Sequential(nn.Linear(32*32*n_features, 2048),
                                nn.ReLU(),
                                nn.Linear(2048, 512),
                                nn.ReLU(),
                                nn.Linear(512,10),
                                nn.Softmax(dim=1))
        
    def forward(self, x):
        x = self.res_blocks(x)
        #reshape x so it becomes flat, except for the first dimension (which is the minibatch)
        x = x.view(x.size(0), -1)
        out = self.fc(x)
        return out

Let's train our new ResNet!

In [ ]:
model = ResNet(3, 8)
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
out_dict = train(model, optimizer)



Do you get nan loss at some point during training? 
This can be caused by the numerical instability of using softmax and log as two functions. 
* Change your network and loss to use a layer that combines the softmax log into one such as `nn.LogSoftmax`. You can also use `nn.CrossEntropyLoss` which also integrates `nn.NLLLoss`.